In [1]:
import sys
import os
sys.path.append("./../bin/Release")
sys.path.append("./../lib/Release")
sys.path.append("./../")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pandas as pd
from pyminion.test import MWUT
from scipy.stats import rankdata

In [2]:
goptimum_cec22 = np.array([300, 400, 600, 800, 900, 1800, 2000, 2200, 2300,  2400,2600,2700])
goptimum_cec20 = np.array([100, 1100,700, 1900, 1700, 1600, 2100, 2200, 2400, 2500])
goptimum_cec17 = np.array([100*i for i in range(1, 31)])
goptimum_cec14= goptimum_cec17
yearToMin = {2017:goptimum_cec17, 2014 : goptimum_cec14, 2020 : goptimum_cec20, 2022:goptimum_cec22}
yearToNfuncs = {2017:30, 2014:30, 2020:10, 2022:12}
resFolder = "./Results/"

# Scoring the Performance of DE Variants on Multiple CEC Problems (Higher is better. Maximum score =100)

### CEC 2014, 2017, 2019, 2020, 2022

In [ ]:
algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
year=2017

multiplier = 5000
dimToYear = {
    2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier}, 
    #2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier}, 
    2014 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier }, 
    #2020 : {5 : 5*multiplier, 10:10*multiplier, 15:15*multiplier, 20:20*multiplier },
    2020 : {5 : 50000, 10:1000000, 15:3000000, 20:10000000 },
    #2022 : { 10:10*multiplier,  20:20*multiplier },
    2022 : { 10:200000,  20:1000000 }
}
dimMaxEvals = dimToYear[year]

matrices = {}
for algo in algos : 
    matList = []
    for dim, maxevals in dimMaxEvals.items() : 
        filename = resFolder+"results_"+str(year)+"_"+ algo +"_"+str(dim) +"_"+str(maxevals)+".txt" 
        matList.append(
            np.loadtxt(filename, delimiter='\t')
        )
    matrices[algo] = matList

def calcSE(year) : 
    ret = {}
    for algo in algos :
        se = 0.0 
        coeff=0.1
        Nfuncs = yearToNfuncs[year]
        globMin = yearToMin[year]
        for arr in matrices[algo] : 
            ae = np.mean(np.abs(np.array(arr)-globMin), axis=0) 
            #print(ae)
            for ifunc in range(Nfuncs) : 
                if year==2017 and ifunc==1 : continue
                #if year==2017 and (ifunc in range( 20)) : continue
                se += coeff*ae[ifunc]
            coeff=coeff+0.1
        ret[algo] = se
    return ret
            
def calcRank(arrDict, ideal, mode="mean") : 
    #here arrlist is [[,,,], [,...]]
    experiments = []
    for algo, arr in arrDict.items() : 
        if year == 2020 : arr = arr[:30]
        experiments.append( np.abs(np.array(arr)-ideal) )
    try : 
        experiments = np.array(experiments)
    except : 
        for e in experiments : print(len(e))
        print(ideal, experiments)
        raise Exception()
    
    if mode=="ranksum" : 
        flattened_data = experiments.flatten()
        ranks = rankdata(flattened_data)
        ranked_experiments = ranks.reshape(experiments.shape)
        total_ranks = np.sum(ranked_experiments, axis=1)
    elif mode == "mean" : 
        total_ranks = np.mean(experiments, axis=1)
    elif mode =="median" :
        total_ranks = np.median(experiments, axis=1)
    else : raise Exception("Uknown mode.")
    
    final_rank = rankdata(total_ranks)
    return dict(zip(arrDict.keys(), final_rank))

def calcSR(year) : 
    Nfuncs = yearToNfuncs[year]
    minimum = yearToMin[year]
    ret = {}
    for algo in algos :
        sr = 0.0 
        coeff=0.1
        Nfuncs = yearToNfuncs[year]
        for idim, arr in enumerate(matrices[algo]) : 
            for ifunc in range(Nfuncs) : 
                if year==2017 and ifunc==1 : continue
                #if year==2017 and (ifunc in range( 20)) : continue
                arrDict= {}
                for algo_ in algos : 
                    arrDict[algo_] = matrices[algo_][idim][:, ifunc]
                rankDict = calcRank(arrDict, minimum[ifunc])
                sr += coeff*rankDict[algo]
            coeff=coeff+0.1
        ret[algo] = sr
    return ret

    
SR = calcSR(year)
SE = calcSE(year)
print(SR)
SEmin = np.min(list(SE.values()))
SRmin = np.min(list(SR.values()))
score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
final = {algo:score1[algo]+score2[algo] for algo in algos}
print("Score1 : ", score1)
print("Score2 : ", score2)
final

In [ ]:
algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
year=2022

multiplierList = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
score1List = []
score2List = []
scoreTotList = []
for multiplier in multiplierList :
    dimToYear = {
        2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier}, 
        #2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier}, 
        2014 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier }, 
        2020 : {5 : 5*multiplier, 10:10*multiplier, 15:15*multiplier, 20:20*multiplier },
        #2020 : {5 : 50000, 10:1000000, 15:3000000, 20:10000000 },
        2022 : { 10:10*multiplier,  20:20*multiplier },
        #2022 : { 10:200000,  20:1000000 }
    }
    if (multiplier ==0) : 
        dimToYear [2020] ={5 : 50000, 10:1000000, 15:3000000, 20:10000000 }
        dimToYear [2022] = { 10:200000,  20:1000000 }
        
    dimMaxEvals = dimToYear[year]

    matrices = {}
    for algo in algos : 
        matList = []
        for dim, maxevals in dimMaxEvals.items() : 
            filename = resFolder+"results_"+str(year)+"_"+ algo +"_"+str(dim) +"_"+str(maxevals)+".txt" 
            matList.append(
                np.loadtxt(filename, delimiter='\t')
            )
        matrices[algo] = matList

    def calcSE(year) : 
        ret = {}
        for algo in algos :
            se = 0.0 
            coeff=0.1
            Nfuncs = yearToNfuncs[year]
            globMin = yearToMin[year]
            for arr in matrices[algo] : 
                ae = np.mean(np.abs(np.array(arr)-globMin), axis=0) 
                #print(ae)
                for ifunc in range(Nfuncs) : 
                    if year==2017 and ifunc==1 : continue
                    se += coeff*ae[ifunc]
                coeff=coeff+0.1
            ret[algo] = se
        return ret
                
    def calcRank(arrDict, ideal, mode="mean") : 
        #here arrlist is [[,,,], [,...]]
        experiments = []
        for algo, arr in arrDict.items() : 
            if year == 2020 : arr = arr[:30]
            experiments.append( np.abs(np.array(arr)-ideal) )
        try : 
            experiments = np.array(experiments)
        except : 
            for e in experiments : print(len(e))
            print(ideal, experiments)
            raise Exception()
        
        if mode=="ranksum" : 
            flattened_data = experiments.flatten()
            ranks = rankdata(flattened_data)
            ranked_experiments = ranks.reshape(experiments.shape)
            total_ranks = np.sum(ranked_experiments, axis=1)
        elif mode == "mean" : 
            total_ranks = np.mean(experiments, axis=1)
        elif mode =="median" :
            total_ranks = np.median(experiments, axis=1)
        else : raise Exception("Uknown mode.")
        
        final_rank = rankdata(total_ranks)
        return dict(zip(arrDict.keys(), final_rank))

    def calcSR(year) : 
        Nfuncs = yearToNfuncs[year]
        minimum = yearToMin[year]
        ret = {}
        for algo in algos :
            sr = 0.0 
            coeff=0.1
            Nfuncs = yearToNfuncs[year]
            for idim, arr in enumerate(matrices[algo]) : 
                for ifunc in range(Nfuncs) : 
                    if year==2017 and ifunc==1 : continue
                    arrDict= {}
                    for algo_ in algos : 
                        arrDict[algo_] = matrices[algo_][idim][:, ifunc]
                    rankDict = calcRank(arrDict, minimum[ifunc])
                    sr += coeff*rankDict[algo]
                coeff=coeff+0.1
            ret[algo] = sr
        return ret
    

    SR = calcSR(year)
    SE = calcSE(year)
    SEmin = np.min(list(SE.values()))
    SRmin = np.min(list(SR.values()))
    score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
    score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
    final = {algo:score1[algo]+score2[algo] for algo in algos}
    score1List.append(score1)
    score2List.append(score2)
    scoreTotList.append(final)

def plot(resList) : 
    algoRes = {algo : [res[algo] for res in resList] for algo in algos}
    xlist = multiplierList
    #xlist = [500, 1000, 5000, 10000]
    plt.figure(figsize=(5,3))
    for algo, res in algoRes.items() : 
        #print(res)
        plt.plot(xlist, res, label=algo)
        plt.scatter(xlist, res)
    plt.legend() 
    #plt.xscale("log")
    plt.show()

plot(score1List)
plot(score2List)
plot(scoreTotList)

### CEC 2011

In [ ]:
algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
matrices = {}
for algo in algos : 
    filename = resFolder+ algo + "_cec2011_31_full_2"+".txt" 
    matrices[algo] = np.loadtxt(filename)
minimum = []
for ifunc in range(20) : 
    arr = np.array([])
    for algo in algos :
        newarr = matrices[algo][:, ifunc]
        arr = np.concatenate((arr, newarr))
    minimum.append(np.min(arr))

print("Minimum : ", minimum)
def calcSE( relative = True) : 
    ret = {}
    for algo in algos :
        se = 0.0 
        for ifunc in range(20) : 
            if not relative :
                se +=  np.mean(matrices[algo][:, ifunc]-minimum[ifunc])
            else : 
                if minimum[ifunc]==0.0 : se +=  np.mean(matrices[algo][:, ifunc])
                else : se +=  np.mean( (matrices[algo][:, ifunc]-minimum[ifunc])/np.abs(minimum[ifunc]))
        ret[algo] = se
    return ret
            
def calcRank(arrDict, ideal, mode="mean") : 
    #here arrlist is [[,,,], [,...]]
    experiments = []
    for algo, arr in arrDict.items() : 
        experiments.append( np.abs(np.array(arr)-ideal) )
    try : 
        experiments = np.array(experiments)
    except : 
        for e in experiments : print(len(e))
        print(ideal, experiments)
        raise Exception()
    if mode=="ranksum" :
        flattened_data = experiments.flatten()
        ranks = rankdata(flattened_data)
        ranked_experiments = ranks.reshape(experiments.shape)
        total_ranks = np.sum(ranked_experiments, axis=1)
    elif mode == "mean" : 
        total_ranks = np.mean(experiments, axis=1)
    elif mode =="median" :
        total_ranks = np.median(experiments, axis=1)
    else : raise Exception("Uknown mode.")
    final_rank = rankdata(total_ranks)
    return dict(zip(arrDict.keys(), final_rank))

def calcSR() : 
    ret = {}
    arr = []
    for algo in algos :
        sr = 0.0 
        for ifunc in range(20) : 
            arrDict= {}
            for algo_ in algos : 
                arrDict[algo_] = matrices[algo_][:, ifunc]
            rankDict = calcRank(arrDict, minimum[ifunc])
            if algo=="LSHADE" : 
                #print("Function number : ", ifunc, " : ", rankDict)
                arr.append(list(rankDict.values()))
            sr += rankDict[algo]
        ret[algo] = sr
    print(np.sum(arr, axis=0))
    return ret

    
SR = calcSR()
SE = calcSE()

SEmin = np.min(list(SE.values()))
SRmin = np.min(list(SR.values()))
score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
final = {algo:score1[algo]+score2[algo] for algo in algos}
print("Score1 : ", score1)
print("Score2 : ", score2)
final

# CEC 2011

In [ ]:
filename = resFolder+"NLSHADE_RSP_cec2011_31_full_2.txt"
data = np.loadtxt(filename)
filename = resFolder+"ARRDE_cec2011_31_full_2.txt"
data1 = np.loadtxt(filename)
filename = resFolder+"j2020_cec2011_31_full_2.txt"
data2 = np.loadtxt(filename)
filename = resFolder+"LSHADE_cec2011_31_full_2.txt"
data3 = np.loadtxt(filename)
filename = resFolder+"LSRTDE_cec2011_31_full_2.txt"
data5 = np.loadtxt(filename)

denom = np.abs(minimum)
for i in range(len(denom)) : 
    if denom[i] == 0.0 : denom[i]=1
denom =1.0
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean((data-minimum)/denom, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-minimum)/denom, axis=0), 
    "j2020" : np.mean(np.abs(data2-minimum)/denom, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-minimum)/denom, axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-minimum)/denom, axis=0), 
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))


print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(20)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(20)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(20)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(20)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
display(df)


In [ ]:
algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
matrices = {}
for algo in algos : 
    filename = resFolder+ algo + "_cec2011_31_full"+".txt" 
    matrices[algo] = np.loadtxt(filename)
minimum = []
for ifunc in range(20) : 
    arr = np.array([])
    for algo in algos :
        newarr = matrices[algo][:, ifunc]
        arr = np.concatenate((arr, newarr))
    minimum.append(np.min(arr))


def calcSE() : 
    ret = {}
    for algo in algos :
        se = 0.0 
        for ifunc in range(20) : 
            se +=  np.mean(np.abs(matrices[algo][:, ifunc]-minimum[ifunc]))
        ret[algo] = se
    return ret
            
def calcRank(arrDict, ideal) : 
    #here arrlist is [[,,,], [,...]]
    experiments = []
    for algo, arr in arrDict.items() : 
        experiments.append( np.abs(np.array(arr)-ideal) )
    
    experiments = np.array(experiments)
    flattened_data = experiments.flatten()
    ranks = rankdata(flattened_data)
    ranked_experiments = ranks.reshape(experiments.shape)
    total_ranks = np.sum(ranked_experiments, axis=1)
    final_rank = rankdata(total_ranks)
    return dict(zip(arrDict.keys(), final_rank))

def calcSR() : 
    ret = {}
    for algo in algos :
        sr = 0.0 
        for ifunc in range(20) : 
            arrDict= {}
            for algo_ in algos : 
                arrDict[algo_] = matrices[algo_][:, ifunc]
            rankDict = calcRank(arrDict, minimum[ifunc])
            sr += rankDict[algo]
        ret[algo] = sr
    return ret

    
SR = calcSR()
SE = calcSE()
SEmin = np.min(list(SE.values()))
SRmin = np.min(list(SR.values()))
score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
final = {algo:score1[algo]+score2[algo] for algo in algos}
print("Score1 : ", score1)
print("Score2 : ", score2)
final

# CEC 2017

In [ ]:
maxevals=100000
filename = resFolder+"results_2017_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0), 
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))



print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
display(df)


In [ ]:
maxevals=30000
filename = resFolder+"results_2017_NLSHADE_RSP_30_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_30_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_30_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_30_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_30_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=50000
filename = resFolder+"results_2017_NLSHADE_RSP_50_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_50_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_50_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_50_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_50_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 

    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals = 100000
filename = resFolder+"results_2017_NLSHADE_RSP_100_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_100_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_100_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_100_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_100_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

# CEC 2020

In [ ]:
maxevals=5000
filename = resFolder+"/results_2020_NLSHADE_RSP_5_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_5_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_5_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_5_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_5_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=100000
filename = resFolder+"results_2020_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df


In [ ]:
maxevals=150000
filename = resFolder+"results_2020_NLSHADE_RSP_15_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_15_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_15_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_15_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_15_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')


df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=200000
filename = resFolder+"results_2020_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_20_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
     "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

# CEC 2022


## 10D

In [ ]:
maxevals=100000
filename = resFolder+"results_2022_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

## 20D


In [ ]:
maxevals=1000000
filename = resFolder+"results_2022_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSHADE_20_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
# 